In [47]:
from src.vectorize import *
from src.lda import *
import pandas as pd
import numpy as np
from keras.utils import to_categorical
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import random

# Import des données et pré-processing

On importe 10 000 articles Wikipedia.

In [48]:
data = pd.read_csv("C:\\Users\\yanis\\OneDrive\\Documents\\ENSAE 3A\\Sequential MC\\SSM-PROJECT\\wiki_data.csv",
                    encoding='utf-8',
                    delimiter=";")
del data["Unnamed: 0"]
display(data.head())
print("Number of articles: {}".format(len(data)))

,Text,Title
0,"Joseph Harold Greenberg (May 28, 1915 – May 7,...",Joseph Greenberg
1,"Pauline Donalda, (March 5, 1882 – October 22,...",Pauline Donalda
2,This is a list of German football transfers in...,List of German football transfers summer 2017
3,"Lester Hudson III (born August 7, 1984) is an ...",Lester Hudson
4,"Monique Ganderton (born August 6, 1980) is a C...",Monique Ganderton


Number of articles: 10000


1. On tokenize chaque article: "Joseph Harold Greenberg" devient ["Jospeh", "Harold", "Greenberg"]
2. On filtre les articles: on retire les articles composés de moins de 500 mots
3. On vectorise le texte: on associe à chaque mot son indice dans le vocabulaire général.

Exemple: si tous les articles peuvent contenir comme mots: ["pomme", "poire", "chocolat", "eau", "banane"], l'article ["chocolat", "banane", "pomme"] devient [2, 4, 0]

In [49]:
vectorized_data=vectorize_data(data=data, min_number_words=500)
display(vectorized_data.head())

,Text,Title,tokenized_text,vectorized_text
0,"Joseph Harold Greenberg (May 28, 1915 – May 7,...",Joseph Greenberg,"[Joseph, Harold, Greenberg, (, May, 28, ,, 191...","[68836, 61034, 58682, 1182, 80975, 13183, 1242..."
3,"Lester Hudson III (born August 7, 1984) is an ...",Lester Hudson,"[Lester, Hudson, III, (, born, August, 7, ,, 1...","[75482, 63781, 64659, 1182, 134097, 26662, 181..."
4,"Monique Ganderton (born August 6, 1980) is a C...",Monique Ganderton,"[Monique, Ganderton, (, born, August, 6, ,, 19...","[84017, 56054, 1182, 134097, 26662, 17386, 124..."
6,The white bikini of Ursula Andress (also known...,White bikini of Ursula Andress,"[The, white, bikini, of, Ursula, Andress, (, a...","[115808, 188693, 133263, 165140, 120335, 24145..."
15,"""Breakout"" is a single from British act Swing ...",Breakout (Swing Out Sister song),"[``, Breakout, '', is, a, single, from, Britis...","[127812, 32984, 6, 156142, 127813, 178055, 149..."


En attendant d'avoir $z^*_{1:T}$, on applique le LDA sur le corpus avec K=50 topics. Chaque mot a alors un poids relativement à chaque topic.

Exemple: le mot banane a les poids suivant: {"topic 1": 0.3, "topic 2": 0.1, "topic 3": 0.05,...}. 

On associe au mot le topic avec le poids le plus élevé. En l'occurence 1. De cette façon, on crée index_topic=$(z_1,...,z_T)$

In [50]:
NUM_TOPICS=50
lda=LDA(num_topics=NUM_TOPICS, random_state=123)
vectorized_data_lda=lda.run(vectorized_data)
display(vectorized_data_lda.head())

,Text,Title,tokenized_text,vectorized_text,index_topic
0,"Joseph Harold Greenberg (May 28, 1915 – May 7,...",Joseph Greenberg,"[Joseph, Harold, Greenberg, (, May, 28, ,, 191...","[68836, 61034, 58682, 1182, 80975, 13183, 1242...","[21, 21, 24, 21, 18, 18, 7, 18, 6, 18, 21, 7, ..."
3,"Lester Hudson III (born August 7, 1984) is an ...",Lester Hudson,"[Lester, Hudson, III, (, born, August, 7, ,, 1...","[75482, 63781, 64659, 1182, 134097, 26662, 181...","[21, 16, 18, 21, 11, 39, 21, 7, 16, 21, 10, 15..."
4,"Monique Ganderton (born August 6, 1980) is a C...",Monique Ganderton,"[Monique, Ganderton, (, born, August, 6, ,, 19...","[84017, 56054, 1182, 134097, 26662, 17386, 124...","[30, 30, 21, 11, 39, 27, 7, 16, 21, 10, 7, 1, ..."
6,The white bikini of Ursula Andress (also known...,White bikini of Ursula Andress,"[The, white, bikini, of, Ursula, Andress, (, a...","[115808, 188693, 133263, 165140, 120335, 24145...","[16, 7, 11, 34, 13, 13, 21, 19, 3, 15, 1, 34, ..."
15,"""Breakout"" is a single from British act Swing ...",Breakout (Swing Out Sister song),"[``, Breakout, '', is, a, single, from, Britis...","[127812, 32984, 6, 156142, 127813, 178055, 149...","[2, 2, 27, 10, 7, 2, 32, 41, 39, 2, 2, 6, 1, 4..."


On applique un padding, i.e une longueur maximale sur les articles ($x$) et les indices de topics ($z$). De façon arbitraire, on fixe le padding à 1000.
* Si l'article/la liste de topics est composé de plus de 1000 caractères alors, on supprime les derniers
* Si l'article/la liste de topics est composé de moins de 1000 caractères alors, on ajoute des 0 à la fin
Cette opération est faite car la taille d'inpput de LSTM est unique

In [51]:
SEQUENCE_LENGTH=1000

def apply_padding(sequence, max_length=SEQUENCE_LENGTH):
    padded_sequence = sequence[:max_length] + [0] * max(0, max_length - len(sequence))
    return padded_sequence

In [52]:
vectorized_data_lda_padding=vectorized_data_lda.copy()
vectorized_data_lda_padding['tokenized_text'] = vectorized_data_lda_padding['tokenized_text'].apply(apply_padding)
vectorized_data_lda_padding['vectorized_text'] = vectorized_data_lda_padding['vectorized_text'].apply(apply_padding)
vectorized_data_lda_padding['index_topic'] = vectorized_data_lda_padding['index_topic'].apply(apply_padding)

On définit un one hot encoding des listes d'indices de topics. Chaque liste $z$, de longueur SEQUENCE_LENGTH=200, est convertie en une matrice de dimensions SEQUENCE_LENGTH*NUM_TOPICS = $1000 \times 50$. L'élement $(i,j)$ de cette matrice vaut 1 si le i-ème topic de la séquence correspond au j-ème topic dans l'ensemble des topics.

In [53]:
def one_hot_encode_list(topic_list, vocab_size):
    return to_categorical(topic_list, num_classes=vocab_size)

In [54]:
array_z=vectorized_data_lda_padding['index_topic'].values
array_z_one_hot_encoded = np.array([one_hot_encode_list(lst, NUM_TOPICS) for lst in array_z])

# LSTM

array_z_one_hot_encoded correspond à l'échantillon de topics $\mathcal{D}_z$ one-hot encodés. Chaque liste de topics $z=(z_1,...,z_T)$ de l'échantillon est une matrice de dimensions $1000 \times 50$. Lorsque l'on aura implémenté le gibbs sampler, cet échantillon n'existera plus: en réalité, les $z$ ne sont pas observés. On remplacera l'échantillon par $z^*_{1:T}$.

On prend un $z$ de l'échantillon en faisant comme si c'était un $z^*$.

In [168]:
z_one_hot=array_z_one_hot_encoded[0]
print(z_one_hot.shape)

(1000, 50)


In [129]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, topics, model_length):
        self.topics=topics
        self.model_length=model_length

    def __len__(self):
        return len(self.topics)-self.model_length

    def __getitem__(self, index):
        input_sequence=torch.tensor(self.topics[index:index+self.model_length, :])
        target_sequence=torch.tensor(self.topics[index+1:index+self.model_length+1, :])

        return input_sequence, target_sequence

In [195]:
import torch.nn.functional as F

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, model_length):
        super(LSTM, self).__init__()
        self.input_size = input_size #dimension d'entrée (NUM_TOPICS)
        self.hidden_size = hidden_size #nombre de neurones de la couche cachée
        self.output_size = output_size #dimension d'outputs (NUM_TOPICS)
        self.model_length=model_length

        self.lstm = nn.LSTM(self.input_size, self.hidden_size, batch_first=True)
        self.fc = nn.Linear(self.hidden_size, self.output_size)
        # self.softmax = nn.Softmax(dim=1) 

    def forward(self, x, prev_state):
        output, state = self.lstm(x, prev_state)
        output=self.fc(output)
        probabilities = F.softmax(output[:, -1, :], dim=1)
        return probabilities, state
    
    def init_state(self):
        return (torch.zeros(1, 1, self.hidden_size), #(NUM_LAYERS, BATCH SIZE, NUM_NEURONES)
                torch.zeros(1, 1, self.hidden_size))
    
    def train_model(self, dataset, optimizer, criterion):
        state_h, state_c = self.init_state()
        self.train()
        for t, (x, y) in enumerate(dataset):
            optimizer.zero_grad()
            softmax , (state_h, state_c) = self(x, (state_h, state_c)) #softmax= p(z_{t+1}|z_1:t)
            loss = criterion(softmax, y[:, -1, :])
            state_h = state_h.detach()
            state_c = state_c.detach()

            loss.backward()
            optimizer.step()
            print({'loss': loss.item() })
    
    def predict_next_probability(self, input_sequence):
        state_h, state_c = self.init_state()

        # Forward pass jusqu'à t-1
        for t in range(len(input_sequence)):
            input_t = input_sequence[t].unsqueeze(0).unsqueeze(0)
            _, (state_h, state_c) = self(input_t, (state_h, state_c))

        # Obtenez les probabilités pour x_t
        input_t = input_sequence[-1].unsqueeze(0).unsqueeze(0)
        probabilities, _ = self(input_t, (state_h, state_c))

        return probabilities


In [196]:
MODEL_LENGTH=200
dataset=Dataset(topics=z_one_hot, model_length=MODEL_LENGTH)
dataloader = DataLoader(dataset, batch_size=1)
print(dataset.__getitem__(0)[0].shape)
print(dataset.__getitem__(0)[1].shape)

torch.Size([200, 50])
torch.Size([200, 50])


In [197]:
HIDDEN_SIZE=64
model=LSTM(input_size=NUM_TOPICS, hidden_size=HIDDEN_SIZE, output_size=NUM_TOPICS, model_length=MODEL_LENGTH)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [198]:
model.train_model(dataloader, optimizer, criterion)

{'loss': 3.913480281829834}
{'loss': 3.913555145263672}
{'loss': 3.9127607345581055}
{'loss': 3.9104163646698}
{'loss': 3.9094271659851074}
{'loss': 3.9140820503234863}
{'loss': 3.9120399951934814}
{'loss': 3.9104113578796387}
{'loss': 3.9092724323272705}
{'loss': 3.9130101203918457}
{'loss': 3.909694194793701}
{'loss': 3.9124979972839355}
{'loss': 3.909458875656128}
{'loss': 3.911994695663452}
{'loss': 3.9093897342681885}
{'loss': 3.910706043243408}
{'loss': 3.913931369781494}
{'loss': 3.9100348949432373}
{'loss': 3.910282850265503}
{'loss': 3.909198760986328}
{'loss': 3.9102394580841064}
{'loss': 3.9092965126037598}
{'loss': 3.9106764793395996}
{'loss': 3.9104933738708496}
{'loss': 3.9138283729553223}
{'loss': 3.913520097732544}
{'loss': 3.912780284881592}
{'loss': 3.9138669967651367}
{'loss': 3.913709878921509}
{'loss': 3.9098753929138184}
{'loss': 3.91274356842041}
{'loss': 3.9122300148010254}
{'loss': 3.9126901626586914}
{'loss': 3.9134511947631836}
{'loss': 3.9100000858306885}
{'

In [205]:
z_one_hot_tensor = torch.FloatTensor(z_one_hot)
z_one_hot_tensor.shape

torch.Size([1000, 50])

In [201]:
model.predict_next_probability(z_one_hot_tensor)

tensor([[0.0186, 0.0222, 0.0197, 0.0190, 0.0187, 0.0223, 0.0202, 0.0232, 0.0198,
         0.0213, 0.0214, 0.0211, 0.0176, 0.0197, 0.0220, 0.0220, 0.0188, 0.0203,
         0.0192, 0.0192, 0.0191, 0.0216, 0.0177, 0.0175, 0.0184, 0.0178, 0.0226,
         0.0186, 0.0180, 0.0208, 0.0196, 0.0194, 0.0219, 0.0195, 0.0186, 0.0179,
         0.0197, 0.0215, 0.0183, 0.0222, 0.0205, 0.0193, 0.0196, 0.0206, 0.0180,
         0.0185, 0.0217, 0.0225, 0.0212, 0.0210]], grad_fn=<SoftmaxBackward0>)

# SSM

Pour illustrer le fonctionnement, on prend le premier article vectorizé associé à la première liste de topics vectorizée. En théorie, cette liste de topics devrait être $z^*$. Pas besoin de one-hot-encoding. Le but est de trouver le maximum de vraisemblance $(\hat{\phi_z})$.

$\phi_z$ est une matrice de taille NUM_WORDS*NUM_TOPICS. $\phi_z[i,j]=$ Pr(mot i | topic j). Le MLE est donné par la contrepartie empirique de ces probabilités: $\frac{\text{Nombre de mots i dans l'échantillon associés au topic j}}{\sum_{i}\text{Nombre de mots i dans l'échantillon associés au topic j}}$

In [ ]:
array_z=vectorized_data_lda_padding['index_topic'].values
z=np.array(array_z[0])
print(z.shape)
print(z)

(200,)
[21 21 24 21 18 18  7 18  6 18 21  7 16 21 28 15  7 22  7  3 19 18 29  7
 32  3 38 34  1  7 28 34 24 21 16 34 15 34 34 21 29 10 16 29 16 21 21 24
 28 11 29 18 18  7 18 44  5 15 18 16  7 18 21 21 32 18  3 29 28  2 21 15
  1 20 34 27  7 29  3  7 16 33 18 38  3 21 34 28 44  1  1 16  3  3 29 15
 21  1  1 43 43  7 29 15 44  1  7 22  1 10 19  7  2 43 21 34 43 19 34 34
 18 18 21 21  1 29  1 28  7 29 34  7 43 34  3 18 38 32  7 34 24 21 16  2
 32 38 34 18 41  7 29 28 32 15  7 43 43  3  6 21 38 19 43 34 18 34 21  1
  1 43 34 29 43 34  7 24 29 38 14  1 38 20 34 41  7  1 29 32  1 38 24 21
 16 34 34 29 34 22 28  1]


In [ ]:
array_x=vectorized_data_lda_padding['vectorized_text'].values
x=np.array(array_x[0])
print(x.shape)
print(x)

(200,)
[ 68836  61034  58682   1182  80975  13183   1242   8867 192633  80975
  18147   1242  11580   1183 188139 129921  23715 158664   1242 157188
 159730 148870 152952 189165 138610 158665 185521 130019 183121 150335
 137193 165140 157659   1434  75829  48696 158437 130019 144754   1182
  79074 179225  20189  42585  11647   1183  68836  58682 188139 134097
 165380  80975  13183   1242   8867 183896  68370 166977 154473  33460
   1242  87590 126649   1434  62734 148179 151223 155610 188139 162918
   1434  26358 183121 128842 165140   5402   1242 152281 150196 127813
 168216 138612 154473 111425  60448   1434  61487 139234 183896 168675
 183121 168216 149491 183590 152952 158437   1434  22055 148116 152752
 175946   1242 152281 141424 183896 171364 127813 175938 135670 172111
 183103 127813 162928 165428   1434  61487 145676 131477  40660 120058
 154473  87590 126649   1434  47977 152952 176849 189547   1242 152281
 131571 127813 137172 182536 135001  54494  31743 138610  23715  65796

In [ ]:
vocab = set()

for token in vectorized_data_lda["tokenized_text"]:
    vocab.update([word for word in token])

vocab = sorted(vocab)
NUM_WORDS=len(vocab)
print('Unique words: {}'.format(NUM_WORDS))

Unique words: 193476


In [ ]:
word_topic_counts = np.zeros((NUM_WORDS, NUM_TOPICS))

for i in range(len(x)):
    word_topic_counts[x[i], z[i]] += 1

MLE_phi_z = word_topic_counts/(np.sum(word_topic_counts, axis=0)+1e-6)
print(MLE_phi_z)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [220]:
def compute_MLE_SSM(x,z, n_vocab, n_topics):
    word_topic_counts = np.zeros((n_vocab, n_topics))

    for i in range(len(x)):
        word_topic_counts[x[i], z[i]] += 1

    MLE_phi_z = word_topic_counts/(np.sum(word_topic_counts, axis=0)+1e-6)
    return MLE_phi_z

## Gibbs

In [219]:
P = 100  # Number of particles
T = 1000  # Number of time steps

z_particles = np.zeros((P, T + 1))  # Initialize particle paths
alpha_particles = np.ones(P) / P  # Initialize normalized weights

for t in range(1, T + 1):
    a1_t_minus_1 = 1
    z1_t = z_particles[0, :t]

    for p in range(P):
        alpha_t_minus_1=
        softmax=model.predict_next_probability(z_particles)